<a href="https://colab.research.google.com/github/EmmaCOo/ADS509_Text_Mining_Final_Project/blob/main/Data__Acquistion_Twitter_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**ADS 509_TEXT MINING_Final_Project : Data Acquisition with Twitter APIs**

###**Emma Oo**


In [ ]:
#install tweepy higher than 4.0
!pip install git+https://github.com/tweepy/tweepy.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-90rx_e_m
  Running command git clone -q https://github.com/tweepy/tweepy.git /tmp/pip-req-build-90rx_e_m


In [ ]:
!pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests-oauthlib, requests, oauthlib
Required-by: 


In [40]:
api_key = 'key' 
api_key_secret = 'key'
bearer_token = 'key'
access_token = 'key'
access_token_secret = 'key'


In [ ]:
# Authenticate
import tweepy
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [ ]:
# for the twitter section
import shutil
import pandas as pd
import tweepy
import os
import datetime
import re
from pprint import pprint

import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

https://blog.quantinsti.com/twitter-api-v2/
(good one I follow)

https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet


In [ ]:
chains = dict()
for handle in ['McDonalds', 'BurgerKing'] :
  user_obj = client.get_user(username=handle,user_fields=["public_metrics"]) 
  chains[handle] = (user_obj.data.id,
                     handle, user_obj.data.public_metrics['followers_count'])
  
for chain, data in chains.items() :
  print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {chain}. ")

It would take 77.80 hours to pull all 4668182 followers for McDonalds. 
It would take 33.99 hours to pull all 2039380 followers for BurgerKing. 


In [ ]:
import os
if not os.path.isdir("twitter") : 
  #shutil.rmtree("twitter/") 
  os.mkdir("twitter")
    

In [ ]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

###**MCDONALDS API PULL FOR FOLLOWERS AND DESCRIPTIONS**

In [ ]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

handles = ['McDonalds']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit= 20):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count','like_count', 'description']
    columns_2 = ['ID']
    mcd = pd.DataFrame(user_data[handle], columns=columns)
    mcd_users = pd.DataFrame(followers_data[handle], columns = columns_2)
  
  
  #Write to the output file
    mcd.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    mcd_users.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': 'FUJA6KK2Q4S1GZZZ'}
{'result_count': 1000, 'next_token': '8093AHJA3GS1GZZZ', 'previous_token': 'RHRM0T7K5R3UEZZZ'}
{'result_count': 1000, 'next_token': 'U6M981Q0EKRHGZZZ', 'previous_token': '7F9SCDM1SF3UEZZZ'}
{'result_count': 1000, 'next_token': 'T9LD0T3GN8R1GZZZ', 'previous_token': '58S8SF75HB4EEZZZ'}
{'result_count': 1000, 'next_token': 'RV1JNNAJ20R1GZZZ', 'previous_token': 'NCJ1S95L8N4UEZZZ'}
{'result_count': 1000, 'next_token': 'RIJ2QU4SECQHGZZZ', 'previous_token': 'THN4PT5CTV4UEZZZ'}
{'result_count': 1000, 'next_token': 'I7GRDILJKOQ1GZZZ', 'previous_token': 'LESCS438HJ5EEZZZ'}
{'result_count': 1000, 'next_token': 'CLSP8TC0TSPHGZZZ', 'previous_token': '69POECE3B75UEZZZ'}
{'result_count': 1000, 'next_token': '0V9FVA5MAOPHGZZZ', 'previous_token': '943UH04Q236EEZZZ'}
{'result_count': 1000, 'next_token': 'ISK4IH2CKCP1GZZZ', 'previous_token': 'USICRHQCL76EEZZZ'}
{'result_count': 1000, 'next_token': 'DR0GRDDAVOOHGZZZ', 'previous_token': 'EPO54P8NBN6U

{'result_count': 1000, 'next_token': '4EH416IISKNHGZZZ', 'previous_token': 'T1BIDAO5P77UEZZZ'}
{'result_count': 1000, 'next_token': 'U7MGV92TL4NHGZZZ', 'previous_token': 'NKAUF5M43B8EEZZZ'}
{'result_count': 1000, 'next_token': 'F91VH9LFSON1GZZZ', 'previous_token': 'E0KS1TMSAR8EEZZZ'}
{'result_count': 1000, 'next_token': 'D38VNQ2028N1GZZZ', 'previous_token': 'JA5A19SL3B8UEZZZ'}
{'result_count': 1000, 'next_token': 'FTFK8I3CASMHGZZZ', 'previous_token': 'VH5SML3PTR8UEZZZ'}
{'result_count': 1000, 'next_token': 'KN0SECMPJ8M1GZZZ', 'previous_token': 'QGDTQ1MUL39EEZZZ'}
0:15:03.747301


In [ ]:
mcd.shape

(19999, 8)

In [ ]:
mcd_df = mcd[mcd["description"].str.strip().str.len()>0]
mcd_df.shape

(9269, 8)

In [ ]:
#check any duplicates
print((mcd_df.duplicated()).sum())

0


###**BURGER KING API PULL FOR FOLLOWERS AND DESCRIPTIONS**

In [ ]:
handles = ['BurgerKing']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit= 20):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count','like_count', 'description']
    columns_2 = ['ID']
    bgk = pd.DataFrame(user_data[handle], columns=columns)
    bgk_users = pd.DataFrame(followers_data[handle], columns = columns_2)
  
  
  #Write to the output file
    bgk.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    bgk_users.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': '6QSS6CLEGGSHGZZZ'}
{'result_count': 1000, 'next_token': 'T4SNT93NAGSHGZZZ', 'previous_token': 'RM4K5PQVFF3EEZZZ'}
{'result_count': 1000, 'next_token': 'G7CHGS6358SHGZZZ', 'previous_token': 'SSR866SILF3EEZZZ'}
{'result_count': 1000, 'next_token': '0G4CTKC4V4S1GZZZ', 'previous_token': '7MNMFQPVQN3EEZZZ'}
{'result_count': 1000, 'next_token': 'DO686IFNP8S1GZZZ', 'previous_token': 'IS5ROVC00R3UEZZZ'}
{'result_count': 1000, 'next_token': 'CLIVT30RJ8S1GZZZ', 'previous_token': '0JN1GNGN6N3UEZZZ'}
{'result_count': 1000, 'next_token': 'D1G2GVC9BSS1GZZZ', 'previous_token': '9MQGE3V5CN3UEZZZ'}
{'result_count': 1000, 'next_token': '724LPOB54CS1GZZZ', 'previous_token': '44FC2CRTK33UEZZZ'}
{'result_count': 1000, 'next_token': 'QR25KGSDV0RHGZZZ', 'previous_token': 'M670AIKSRJ3UEZZZ'}


{'result_count': 1000, 'next_token': '1VRB2GO8P8RHGZZZ', 'previous_token': 'L9N9NSC30V4EEZZZ'}
{'result_count': 1000, 'next_token': '9FCFVTETJKRHGZZZ', 'previous_token': 'MT8NIBGA6R4EEZZZ'}
{'result_count': 1000, 'next_token': 'ST8KS8CMBSRHGZZZ', 'previous_token': 'F85I82P2CB4EEZZZ'}
{'result_count': 1000, 'next_token': 'ANSIVS9O40RHGZZZ', 'previous_token': 'AP013E3EK34EEZZZ'}
{'result_count': 1000, 'next_token': '024Q62UITCR1GZZZ', 'previous_token': 'SKGJU8MSRV4EEZZZ'}
{'result_count': 1000, 'next_token': '5OI3B3KTO8R1GZZZ', 'previous_token': 'CQNIC31S2J4UEZZZ'}
{'result_count': 1000, 'next_token': '36IR9TDLJ0R1GZZZ', 'previous_token': '6SORCNBF7N4UEZZZ'}
{'result_count': 1000, 'next_token': '3P1I7DDGD8R1GZZZ', 'previous_token': 'NQSL2GIACV4UEZZZ'}
{'result_count': 1000, 'next_token': 'PTJHOACT80R1GZZZ', 'previous_token': '7RM32UIUIN4UEZZZ'}
{'result_count': 1000, 'next_token': 'GTDAFENQ2KR1GZZZ', 'previous_token': '59LPDRR8NV4UEZZZ'}
{'result_count': 1000, 'next_token': 'RM14RB26SSQH

In [ ]:
bgk_df = bgk[bgk["description"].str.strip().str.len()>0]
bgk_df.shape

(7011, 8)

In [ ]:
#check any duplicates
print((bgk_df.duplicated()).sum())

0


###**WENDYS API PULL FOR FOLLOWERS AND DESCRIPTIONS**




In [ ]:
handles = ['Wendys']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit= 20):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count','like_count', 'description']
    columns_2 = ['ID']
    wd = pd.DataFrame(user_data[handle], columns=columns)
    wd_users = pd.DataFrame(followers_data[handle], columns = columns_2)
  
  
  #Write to the output file
    wd.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    wd_users.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': 'S61KTDUNQKS1GZZZ'}
{'result_count': 1000, 'next_token': 'LF4IOIDH3KS1GZZZ', 'previous_token': 'DLSN8TT95B3UEZZZ'}
{'result_count': 1000, 'next_token': 'D1E4EP37F0RHGZZZ', 'previous_token': '8G8H90QQSB3UEZZZ'}
{'result_count': 1000, 'next_token': 'N3CHBGLFNCR1GZZZ', 'previous_token': 'NUSPG5LKGV4EEZZZ'}


{'result_count': 1000, 'next_token': '7N58IN7S24R1GZZZ', 'previous_token': '5V3KR1348J4UEZZZ'}
{'result_count': 1000, 'next_token': '9J5JIQ7DEGQHGZZZ', 'previous_token': '2R1Q219STR4UEZZZ'}
{'result_count': 1000, 'next_token': 'GGL2589AL8Q1GZZZ', 'previous_token': 'VGC3EMAAHF5EEZZZ'}
{'result_count': 1000, 'next_token': 'B3F3P6JGU4PHGZZZ', 'previous_token': '18R72PNNAN5UEZZZ'}
{'result_count': 1000, 'next_token': '5L20U5V7B0PHGZZZ', 'previous_token': 'CM92A2F61R6EEZZZ'}
{'result_count': 1000, 'next_token': 'QJ7IP39EKKP1GZZZ', 'previous_token': '01GQF3QRKV6EEZZZ'}
{'result_count': 1000, 'next_token': 'HJJRLKMLVSOHGZZZ', 'previous_token': '9OGPKA6IBB6UEZZZ'}
{'result_count': 1000, 'next_token': '4V4T07NNHSOHGZZZ', 'previous_token': 'KDDE45B3037EEZZZ'}
{'result_count': 1000, 'next_token': 'LM788O6KS0O1GZZZ', 'previous_token': 'F6LGHIPFE37EEZZZ'}
{'result_count': 999, 'next_token': 'C8FBNLEA70O1GZZZ', 'previous_token': '80O5NOQP3V7UEZZZ'}
{'result_count': 1000, 'next_token': '6SG76UGRSONHG

In [ ]:
wd_df = wd[wd["description"].str.strip().str.len()>0]
wd_df.shape

(9271, 8)

###**In-N-Out Burger API PULL FOR FOLLOWERS AND DESCRIPTIONS**


In [ ]:
handles = ['innoutburger']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit= 20):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count','like_count', 'description']
    columns_2 = ['ID']
    inn = pd.DataFrame(user_data[handle], columns=columns)
    inn_users = pd.DataFrame(followers_data[handle], columns = columns_2)
  
  
  #Write to the output file
    inn.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    inn_users.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': 'G2LKH2O32CSHGZZZ'}
{'result_count': 1000, 'next_token': 'BSFO4O5JIGS1GZZZ', 'previous_token': 'M4SOJF85TN3EEZZZ'}
{'result_count': 1000, 'next_token': '3NQOS72C2GS1GZZZ', 'previous_token': 'RNBVAS2EDF3UEZZZ'}
{'result_count': 1000, 'next_token': 'D6O3B5H9N8RHGZZZ', 'previous_token': 'C0QIOM66TF3UEZZZ'}
{'result_count': 1000, 'next_token': '68F4CP543GRHGZZZ', 'previous_token': 'GBC8GB7B8N4EEZZZ'}
{'result_count': 1000, 'next_token': '9BUL27QDIKR1GZZZ', 'previous_token': '70PAK2BASF4EEZZZ'}
{'result_count': 1000, 'next_token': '2RL47B5J6OR1GZZZ', 'previous_token': 'FJQL6D68DB4UEZZZ'}
{'result_count': 1000, 'next_token': 'JNCP200VSGQHGZZZ', 'previous_token': '40HPHQJ8P74UEZZZ'}
{'result_count': 1000, 'next_token': 'VCG86RC0GOQHGZZZ', 'previous_token': '9H1T8PNE3F5EEZZZ'}
{'result_count': 1000, 'next_token': 'GE6TIVLO2GQHGZZZ', 'previous_token': 'GSB21AKDF75EEZZZ'}
{'result_count': 1000, 'next_token': '6OO7LQH1G0Q1GZZZ', 'previous_token': '4215623ETF5E

{'result_count': 1000, 'next_token': '7ULDD3531SPHGZZZ', 'previous_token': '2RVV2SH6GV6EEZZZ'}
{'result_count': 1000, 'next_token': 'UB4DL52HKSP1GZZZ', 'previous_token': '6BK4KPKVU36EEZZZ'}
{'result_count': 1000, 'next_token': 'CFLTCQKT98P1GZZZ', 'previous_token': '56BN0RMHB36UEZZZ'}
{'result_count': 1000, 'next_token': 'P7LJQ7DIUOOHGZZZ', 'previous_token': 'NPDLLFTSMN6UEZZZ'}
{'result_count': 1000, 'next_token': 'B0SF7VURM0OHGZZZ', 'previous_token': '5G7NT4QL177EEZZZ'}
{'result_count': 1000, 'next_token': 'S6DN6EQJHGOHGZZZ', 'previous_token': '1JSK5QHG9V7EEZZZ'}
0:25:52.421812


In [ ]:
inn_df = inn[inn["description"].str.strip().str.len()>0]
inn_df.shape

(9877, 8)

###**JACK IN THE BOX API PULL FOR FOLLOWERS AND DESCRIPTIONS

In [ ]:
handles = ['JackBox']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit= 20):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count','like_count', 'description']
    columns_2 = ['ID']
    jkb = pd.DataFrame(user_data[handle], columns=columns)
    jkb_users = pd.DataFrame(followers_data[handle], columns = columns_2)
  
  
  #Write to the output file
    jkb.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    jkb_users.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': 'P2SI51GR60HHGZZZ'}
{'result_count': 1000, 'next_token': 'N9NJSCACGFVHEZZZ', 'previous_token': 'MTAA0J7EPVEEEZZZ'}
{'result_count': 1000, 'next_token': 'A3R25PNLON81EZZZ', 'previous_token': 'V0GLV3L8FO0EGZZZ'}
{'result_count': 1000, 'next_token': 'KLH7FS4D9V51EZZZ', 'previous_token': 'O913OU0R7KNUGZZZ'}
{'result_count': 1000, 'next_token': 'LNOG027QRJ4HEZZZ', 'previous_token': 'FQIT077RM0QUGZZZ'}
{'result_count': 1000, 'next_token': 'S4IB8UO30II1EZZZ', 'previous_token': 'QBGLCH874CREGZZZ'}
{'result_count': 1000, 'next_token': '95JQIOSA8HEHEZZZ', 'previous_token': '9VVJANUT19EEGZZZ'}
{'result_count': 1000, 'next_token': '6NUGI6P4RKJHEZZZ', 'previous_token': 'M99MD7RONEHEGZZZ'}
{'result_count': 1000, 'next_token': 'CV3R0UTNONN1CZZZ', 'previous_token': 'JDF3J3FF7BCEGZZZ'}


{'result_count': 1000, 'next_token': 'RQIOIN0U3EUHCZZZ', 'previous_token': 'P231GPAFF08UIZZZ'}
{'result_count': 1000, 'next_token': '48SIQ8AK0IBHCZZZ', 'previous_token': '8G89GMK7051UIZZZ'}
{'result_count': 1000, 'next_token': 'S4QGHCAL75U1CZZZ', 'previous_token': 'AQ3S81SQ2LKUIZZZ'}
{'result_count': 1000, 'next_token': 'T06EGQ82U9EHCZZZ', 'previous_token': 'DGCF7P0HOU1UIZZZ'}
{'result_count': 1000, 'next_token': 'S7KH2E4R2DCHCZZZ', 'previous_token': 'NJRDTCVK32HEIZZZ'}
{'result_count': 1000, 'next_token': 'RGQA7SV7NPC1CZZZ', 'previous_token': 'TEFNC7BITIJEIZZZ'}
{'result_count': 1000, 'next_token': 'NRUJ0G7IMP0HCZZZ', 'previous_token': 'V75C711S86JUIZZZ'}
{'result_count': 1000, 'next_token': 'I9DIR91M6OK1CZZZ', 'previous_token': 'T4T4JOM296VEIZZZ'}
{'result_count': 1000, 'next_token': 'NIR17071AC91CZZZ', 'previous_token': 'K7VAK1BEPBBUIZZZ'}
{'result_count': 1000, 'next_token': '6KMO6P5KTC01CZZZ', 'previous_token': 'C77TJAQGLRMUIZZZ'}
{'result_count': 1000, 'next_token': 'PHUF4C3MC7K1

In [ ]:
jkb_df = jkb[jkb["description"].str.strip().str.len()>0]
jkb_df.shape

(13800, 8)

In [ ]:
#check any duplicates
print((bgk_df.duplicated()).sum())
print((mcd_df.duplicated()).sum())
print((inn_df.duplicated()).sum())
print((jkb_df.duplicated()).sum())
print((wd_df.duplicated()).sum())

0
0
0
0
0


In [ ]:
#export as csv files
mcd_df.to_csv('mcd.csv', index= False)
wd_df.to_csv('wendy.csv',index=False)
inn_df.to_csv('innout.csv',index=False)
jkb_df.to_csv('jack.csv',index=False)
bgk_df.to_csv('burgerk.csv', index=False)